### MODEL FOR IMU NEURAL NET

In [1]:
from __future__ import print_function
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from IPython.display import display, HTML

from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.layers import BatchNormalization
from keras.optimizers import Adam
from keras import backend as K

### GETTING Data

In [2]:
dfAwake_train = pd.read_csv('Data/training_AData.txt', header=None, delim_whitespace=True)
dfDrowsy_train = pd.read_csv('Data/training_DData.txt', header=None, delim_whitespace=True)
dfAwake_test = pd.read_csv('Data/testing_AData.txt', header=None, delim_whitespace=True)
dfDrowsy_test = pd.read_csv('Data/testing_DData.txt', header=None, delim_whitespace=True)

### Printing Retrieved Data informations

In [3]:
print(dfAwake_train.shape)
print(dfDrowsy_train.shape)
print(dfAwake_test.shape)
print(dfDrowsy_test.shape)

(3240000, 15)
(3240000, 15)
(1620000, 15)
(1620000, 15)


### Dropping Not Needed Columns Keeping IMU

In [4]:
dfAwake_train = dfAwake_train.drop(14,axis = 1)
dfAwake_train = dfAwake_train.drop(dfAwake_train.iloc[:,0:12],axis = 1)
dfDrowsy_train = dfDrowsy_train.drop(14,axis = 1)
dfDrowsy_train = dfDrowsy_train.drop(dfDrowsy_train.iloc[:,0:12],axis = 1)
dfAwake_test = dfAwake_test.drop(14,axis = 1)
dfAwake_test = dfAwake_test.drop(dfAwake_test.iloc[:,0:12],axis = 1)
dfDrowsy_test = dfDrowsy_test.drop(14,axis = 1)
dfDrowsy_test = dfDrowsy_test.drop(dfDrowsy_test.iloc[:,0:12],axis = 1)

In [5]:
print(dfAwake_train.shape)
print(dfDrowsy_train.shape)
print(dfAwake_test.shape)
print(dfDrowsy_test.shape)

(3240000, 2)
(3240000, 2)
(1620000, 2)
(1620000, 2)


### RESHAPING DATA

In [6]:
dfAwake_train = dfAwake_train.to_numpy()
dfDrowsy_train = dfDrowsy_train.to_numpy()
dfAwake_train = dfAwake_train.reshape(3600,900,2)
dfDrowsy_train = dfDrowsy_train.reshape(3600,900,2)
dfAwake_test = dfAwake_test.to_numpy()
dfDrowsy_test = dfDrowsy_test.to_numpy()
dfAwake_test = dfAwake_test.reshape(1800,900,2)
dfDrowsy_test = dfDrowsy_test.reshape(1800,900,2)
print(dfAwake_train.shape)
print(dfDrowsy_train.shape)
print(dfAwake_test.shape)
print(dfDrowsy_test.shape)
# print(dfAwake.head)
# print(dfDrowsy.head)

(3600, 900, 2)
(3600, 900, 2)
(1800, 900, 2)
(1800, 900, 2)


### CREATING LABELS

In [7]:
dfAwake_trainLabels = np.zeros(3600)
print(dfAwake_trainLabels.shape)
dfDrowsy_trainLabels = np.ones(3600)
print(dfDrowsy_trainLabels.shape)
dfAwake_testLabels = np.zeros(1800)
print(dfAwake_testLabels.shape)
dfDrowsy_testLabels = np.ones(1800)
print(dfDrowsy_testLabels.shape)

(3600,)
(3600,)
(1800,)
(1800,)


In [8]:
print(dfAwake_trainLabels)
print(dfDrowsy_trainLabels)
print(dfAwake_testLabels)
print(dfDrowsy_testLabels)

[0. 0. 0. ... 0. 0. 0.]
[1. 1. 1. ... 1. 1. 1.]
[0. 0. 0. ... 0. 0. 0.]
[1. 1. 1. ... 1. 1. 1.]


### Combining Awake Data and Drowsy Data

In [9]:
dftrain = np.concatenate((dfAwake_train, dfDrowsy_train))
dftrainlabels =np.concatenate((dfAwake_trainLabels, dfDrowsy_trainLabels))
print(dftrain.shape)
print(dftrainlabels.shape)
print(dftrain)
print(dftrainlabels)

(7200, 900, 2)
(7200,)
[[[-0.01347178 -0.00392533]
  [ 0.00561919  0.00493475]
  [ 0.02176988  0.00053666]
  ...
  [ 0.01136684 -0.00309513]
  [ 0.01403857 -0.00426882]
  [ 0.02121865 -0.00393233]]

 [[ 0.00561919  0.00493475]
  [ 0.02176988  0.00053666]
  [ 0.00405722  0.01380083]
  ...
  [ 0.01403857 -0.00426882]
  [ 0.02121865 -0.00393233]
  [ 0.00454181 -0.01287709]]

 [[ 0.02176988  0.00053666]
  [ 0.00405722  0.01380083]
  [ 0.00771384  0.00374309]
  ...
  [ 0.02121865 -0.00393233]
  [ 0.00454181 -0.01287709]
  [-0.00323185 -0.00783903]]

 ...

 [[-0.00226274  0.00575061]
  [ 0.0075034  -0.00658379]
  [ 0.00472974 -0.00671845]
  ...
  [ 0.00261806  0.00260904]
  [-0.00574393  0.00689247]
  [-0.00291118 -0.00434906]]

 [[ 0.0075034  -0.00658379]
  [ 0.00472974 -0.00671845]
  [-0.01043967  0.00304691]
  ...
  [-0.00574393  0.00689247]
  [-0.00291118 -0.00434906]
  [ 0.0099463   0.00430073]]

 [[ 0.00472974 -0.00671845]
  [-0.01043967  0.00304691]
  [-0.00011717 -0.00286729]
  ...
 

### SHUFFLING DATA

In [10]:
idx = np.random.permutation(len(dftrainlabels))
print(idx)

[3518 7112 5983 ... 4343 3757 7161]


In [11]:
dftrain=dftrain[idx]
dftrainlabels=dftrainlabels[idx]

In [12]:
print(dftrain.shape)
print(dftrainlabels.shape)
print(dftrain)
print(dftrainlabels)

(7200, 900, 2)
(7200,)
[[[-2.90092639e-03 -2.79204333e-03]
  [-3.30853802e-03 -1.30947933e-03]
  [ 9.43122306e-03 -1.04934558e-02]
  ...
  [-1.28770645e-03 -1.24637890e-02]
  [ 1.49313893e-02  1.71731082e-02]
  [ 9.09639256e-03  4.18441115e-03]]

 [[-3.62938499e-03 -2.67135525e-03]
  [-4.69024132e-03  2.23973078e-03]
  [-4.87973737e-03  8.45971046e-03]
  ...
  [-1.35397010e-03  7.48637634e-03]
  [-1.89825274e-03 -4.34819060e-03]
  [ 3.23767986e-03  8.27991046e-03]]

 [[ 2.04966944e-03 -5.09992131e-03]
  [ 3.95544318e-03  1.91967999e-03]
  [-2.12188333e-03  9.64311128e-03]
  ...
  [-7.50871505e-02  3.24829239e-02]
  [-2.57322321e-02 -1.27238666e-01]
  [ 9.01524417e-03 -2.81105568e-01]]

 ...

 [[-2.96369740e-02 -6.82038339e-02]
  [-3.92838959e-02 -5.34148932e-02]
  [-2.50104406e-02 -2.93740584e-02]
  ...
  [-1.95138201e-02  1.29840729e-02]
  [-1.57278366e-03  1.32758077e-02]
  [-2.29421074e-04 -5.26506417e-03]]

 [[ 6.67564242e-02 -4.17301544e-02]
  [ 8.01209341e-02  7.24629362e-03]
  [

In [13]:
dftest = np.concatenate((dfAwake_test, dfDrowsy_test))
dftestlabels =np.concatenate((dfAwake_testLabels, dfDrowsy_testLabels))
print(dftest.shape)
print(dftestlabels.shape)
print(dftest)
print(dftestlabels)

(3600, 900, 2)
(3600,)
[[[ 0.02311828  0.011094  ]
  [ 0.03492036 -0.01857721]
  [ 0.02784405 -0.01861933]
  ...
  [ 0.01446212 -0.01794756]
  [ 0.01531532 -0.01282402]
  [ 0.00547928 -0.0126411 ]]

 [[ 0.03492036 -0.01857721]
  [ 0.02784405 -0.01861933]
  [ 0.00261633 -0.02434868]
  ...
  [ 0.01531532 -0.01282402]
  [ 0.00547928 -0.0126411 ]
  [-0.00173669 -0.01653759]]

 [[ 0.02784405 -0.01861933]
  [ 0.00261633 -0.02434868]
  [ 0.01006408 -0.01847897]
  ...
  [ 0.00547928 -0.0126411 ]
  [-0.00173669 -0.01653759]
  [ 0.01429986  0.03591099]]

 ...

 [[-0.01538968 -0.00381893]
  [-0.01108513  0.00549479]
  [-0.0132084  -0.00062503]
  ...
  [-0.00202253  0.0016088 ]
  [-0.00207898 -0.00485158]
  [ 0.00513121 -0.00311212]]

 [[-0.01108513  0.00549479]
  [-0.0132084  -0.00062503]
  [-0.01549576 -0.01200462]
  ...
  [-0.00207898 -0.00485158]
  [ 0.00513121 -0.00311212]
  [ 0.01296438 -0.00488806]]

 [[-0.0132084  -0.00062503]
  [-0.01549576 -0.01200462]
  [-0.01053267  0.00184589]
  ...
 

In [14]:
print(dftest.shape)
print(dftestlabels.shape)
print(dftrain.shape)
print(dftrainlabels.shape)

(3600, 900, 2)
(3600,)
(7200, 900, 2)
(7200,)


### Set input and output dimensions

In [15]:
input_shape = 900*2
num_classes = 2
# dftrain = dftrain.reshape(dftrain.shape[0],input_shape)
print('dftrain shape:', dftrain.shape)
print('input_shape:', input_shape)

dftrain shape: (7200, 900, 2)
input_shape: 1800


In [16]:
dftrain = dftrain.astype('float32')
dftrainlabels = dftrainlabels.astype('float32')

In [17]:
dftrainlabels_hot = np_utils.to_categorical(dftrainlabels,num_classes)
print('New dfttrainlabels shape: ', dftrainlabels_hot)

New dfttrainlabels shape:  [[1. 0.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [18]:
print('New dfttrainlabels shape: ', dftrainlabels_hot.shape)
print(dftrainlabels)

New dfttrainlabels shape:  (7200, 2)
[0. 1. 1. ... 1. 1. 1.]


#### Evaluating Training Data

In [19]:
model = keras.models.load_model("best_model(imuv3).h5")

test_loss, test_acc = model.evaluate(dftrain, dftrainlabels,verbose=1)

print("Test accuracy", test_acc)
print("Test loss", test_loss)

225/225 [==============================] - 1s 2ms/step - loss: 0.4966 - sparse_categorical_accuracy: 0.7350
Test accuracy 0.7350000143051147
Test loss 0.4965668022632599


### Building the MODEL

In [ ]:
# model_m = Sequential()
# # Remark: since coreml cannot accept vector shapes of complex shape like
# # [80,3] this workaround is used in order to reshape the vector internally
# # prior feeding it into the network
# model_m.add(Reshape((900, 2), input_shape=(input_shape,)))
# model_m.add(Dense(100, activation='relu'))
# # model_m.add(BatchNormalization()) # V2 addition
# model_m.add(Dense(100, activation='relu'))
# # model_m.add(BatchNormalization()) # V2 addition
# model_m.add(Dense(100, activation='relu'))
# # model_m.add(BatchNormalization()) # V2 addition 
# model_m.add(Flatten())
# model_m.add(Dense(num_classes, activation='softmax'))
# print(model_m.summary())

In [ ]:
def make_model(input_shape):
    input_layer = keras.layers.Input(input_shape)

    conv1 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.ReLU()(conv1)

    # conv2 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv1)
    # conv2 = keras.layers.BatchNormalization()(conv2)
    # conv2 = keras.layers.ReLU()(conv2)

    # conv3 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv2)
    # conv3 = keras.layers.BatchNormalization()(conv3)
    # conv3 = keras.layers.ReLU()(conv3)

    gap = keras.layers.GlobalAveragePooling1D()(conv1)

    output_layer = keras.layers.Dense(num_classes, activation="softmax")(gap)

    return keras.models.Model(inputs=input_layer, outputs=output_layer)


model = make_model(input_shape=dftrain.shape[1:])
keras.utils.plot_model(model, show_shapes=True)

### FITTING Deep neural network Model in KERAS

In [ ]:
# callbacks_list = [
#     keras.callbacks.ModelCheckpoint(
#         filepath='best_model(imu).{epoch:02d}-{val_loss:.2f}.h5',
#         monitor='val_loss', save_best_only=True),
#     keras.callbacks.EarlyStopping(monitor='accuracy', patience=1)
# ]



# model_m.compile(loss='categorical_crossentropy',
#                 optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

# # Hyper-parameters
# BATCH_SIZE = 400 # V3 changed to 2160
# EPOCHS = 50


# # Enable validation to use ModelCheckpoint and EarlyStopping callbacks.
# # v1
# history = model_m.fit(dftrain,
#                       dftrainlabels_hot,
#                       batch_size=BATCH_SIZE,
#                       epochs=EPOCHS,
#                       callbacks=callbacks_list,
#                       validation_split=0.2,
#                       verbose=1)

# # # v1 completing all epochs
# # history = model_m.fit(dftrain,
# #                       dftrainlabels_hot,
# #                       batch_size=BATCH_SIZE,
# #                       epochs=EPOCHS,
# #                       validation_split=0.2,
# #                       verbose=1)

# # v3
# # history = model_m.fit(dftrain,
# #                       dftrainlabels_hot,
# #                       batch_size=BATCH_SIZE,
# #                       epochs=EPOCHS,
# #                       shuffle=True,
# #                       validation_split=0.2,
# #                       verbose=1)


In [ ]:
epochs = 100
batch_size = 288

callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model(imu).h5", save_best_only=True, monitor="val_loss"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=5, min_lr=0.0001
    ),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=1),
]
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"],
)
# model.compile(
#     optimizer="adam",
#     loss="categorical_crossentropy",
#     metrics=["accuracy"],
# )
history = model.fit(
    dftrain,
    dftrainlabels,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_split=0.4,
    verbose=1,
)

In [ ]:
# plt.figure(figsize=(6, 4))
# plt.plot(history.history['accuracy'], 'r', label='Accuracy of training data')
# plt.plot(history.history['val_accuracy'], 'b', label='Accuracy of validation data')
# plt.plot(history.history['loss'], 'r--', label='Loss of training data')
# plt.plot(history.history['val_loss'], 'b--', label='Loss of validation data')
# plt.title('Model Accuracy and Loss')
# plt.ylabel('Accuracy and Loss')
# plt.xlabel('Training Epoch')
# plt.ylim(0)
# plt.legend()
# plt.show()

metric = "sparse_categorical_accuracy"
plt.figure()
plt.plot(history.history[metric])
plt.plot(history.history["val_" + metric])
plt.plot(history.history['loss'], 'r--', label='Loss of training data')
plt.plot(history.history['val_loss'], 'b--', label='Loss of validation data')
plt.title("model " + metric)
plt.ylabel(metric, fontsize="large")
plt.xlabel("epoch", fontsize="large")
plt.legend(["train", "val","loss","val_loss"], loc="best")
plt.show()
plt.close()

# Print confusion matrix for training data
y_pred_train = model.predict(dftrain)
# Take the class with the highest probability from the train predictions
max_y_pred_train = np.argmax(y_pred_train, axis=1)
print(classification_report(dftrainlabels, max_y_pred_train))

### Checking against Testing Data

In [ ]:

# Set input_shape / reshape for Keras
# dftest = dftest.reshape(dftest.shape[0], input_shape)

dftest = dftest.astype('float32')
dftestlabels = dftestlabels.astype('float32')

dftestlabels_hot = np_utils.to_categorical(dftestlabels, num_classes)


In [ ]:
print('New dfttrainlabels shape: ', dftestlabels_hot)
print('New dfttrainlabels shape: ', dftestlabels.shape)
print('New dfttrainlabels shape: ', dftestlabels)

In [ ]:
# score = model_m.evaluate(dftest, dftestlabels_hot, verbose=1)

# print('\nAccuracy on test data: %0.2f' % score[1])
# print('\nLoss on test data: %0.2f' % score[0])

In [ ]:
LABELS = ['Awake','Drowsy']

In [ ]:
# metric = "sparse_categorical_accuracy"
# plt.figure()
# plt.plot(history.history[metric])
# plt.plot(history.history["val_" + metric])
# plt.title("model " + metric)
# plt.ylabel(metric, fontsize="large")
# plt.xlabel("epoch", fontsize="large")
# plt.legend(["train", "val"], loc="best")
# plt.show()
# plt.close()s

In [ ]:
model = keras.models.load_model("best_model(imuv3).h5")

test_loss, test_acc = model.evaluate(dftest, dftestlabels,verbose=1)

print("Test accuracy", test_acc)
print("Test loss", test_loss)

In [ ]:
# # Print confusion matrix for training data
# y_pred_train = model.predict(dftest)
# # Take the class with the highest probability from the train predictions
# max_y_pred_train = np.argmax(y_pred_train, axis=1)
# print(classification_report(dftestlabels, max_y_pred_train))

In [ ]:
def show_confusion_matrix(validations, predictions):

    matrix = metrics.confusion_matrix(validations, predictions)
    plt.figure(figsize=(6, 4))
    sns.heatmap(matrix,
                cmap='coolwarm',
                linecolor='white',
                linewidths=1,
                xticklabels=LABELS,
                yticklabels=LABELS,
                annot=True,
                fmt='d')
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

y_pred_test = model.predict(dftest)
# Take the class with the highest probability from the test predictions
max_y_pred_test = np.argmax(y_pred_test, axis=1)
max_y_test = np.argmax(dftestlabels_hot, axis=1)

show_confusion_matrix(max_y_test, max_y_pred_test)

print(classification_report(max_y_test, max_y_pred_test))